x_axis,variable,value
0,X,0
1000,X,1
2000,X,2
3000,X,3
4000,X,4
5000,X,5
6000,X,6
7000,X,7
8000,X,8
9000,X,9


In [ ]:
#App.R

## SETUP

library(shiny)
library(leaflet)
library(ggplot2)
library(dplyr)
library(tidyr)
library(reshape2)

options(scipen=999)

# Data Import

#All Meteorites
meteorites <- read.csv(file='data/results/meteorites_lc.csv')
meteorites <- meteorites[, c(2,3,7,8,6,10,9,18,13:16,12,5)]
colnames(meteorites) <- c('id','Name','Fell_or_Found','Year', 'Mass', 'Longitude', 'Latitude', 'Land_Cover', 'Chondrite', 'Type', 'Level_3', 'Level_4', 'Group','recclass')

#Prob Density Function Land Cover
pdf <- read.csv(file = 'data/results/pdf.csv')
pdfmelt<-melt(pdf,id="x_axis")

LC_names  <- list("Oceans/Water",
                  "Cultivated Land",
                  "Pasture/Land for Grazing",
                  "Tundra",
                  "Wooded Tundra",
                  "Boreal Forest",
                  "Cool Conifer Forest",
                  "Temperate Mixed Forest",
                  "Temperate Decidious Forest",
                  "Grassland/Steppe",
                  "Hot Desert",
                  "Savanna",
                  "Tropical Woodland",
                  "Tropical Forest")

LC_values  <- list("water_pdf",
                   "cultivated_pdf",
                   "pasture_pdf",
                   "tundra_pdf",
                   "wooded_tundra_pdf",
                   "boreal_forest_pdf",
                   "conifer_forest_pdf",
                   "temperate_mixed_forest_pdf",
                   "temperate_decidous_forest_pdf",
                   "grasssland_pdf",
                   "desert_pdf",
                   "savanna_pdf",
                   "tropical_wooded_pdf",
                   "tropical_forest_pdf")

#UI

ui <- fluidPage(
    
    titlePanel("Meteorite Data Analysis"),
    
    fluidRow(
          tabsetPanel(
              tabPanel("Meteorite Map",
                       fluidRow(
                           column(1,(hr)),
                           column(2,
                                  h3("Filters:"),
                                  sliderInput("yearslider",
                                                label = "Years",
                                                min = 860, 
                                                max = 2021,
                                                value = c(860,2021),
                                                sep=""),
                                  checkboxGroupInput(inputId = "fellfoundcheckbox",
                                                     label = "Fall or Find:",
                                                     inline = TRUE,
                                                     selected = c("Fell", "Found"),
                                                     choiceNames = c("Observed Falling","Found later"),
                                                     choiceValues = c("Fell", "Found")),
                                  checkboxGroupInput(inputId = "typecheckbox",
                                                     label = "Type of Meteorite:",
                                                     inline = TRUE,
                                                     selected = c("Stony","Stony-Iron","Iron"),
                                                     choiceNames = c("Stony","Stony-Iron","Iron"),
                                                     choiceValues= c("Stony","Stony-Iron","Iron")),
                                  checkboxGroupInput(inputId = "chondritecheckbox",
                                                     label = "Chondrite or Achondrite:",
                                                     inline = TRUE,
                                                     selected = c("Chondrite","Achondrite"),
                                                     choiceNames = c("Chondrite","Achondrite"),
                                                     choiceValues = c("Chondrite","Achondrite"))),
                           column(2,(hr)(hr)(hr),
                                  sliderInput("massslider",
                                                label = "Mass (grams)",
                                                min = 0, 
                                                max = 60000000,
                                                value = c(0,60000000)),
                                  checkboxGroupInput(inputId = "meteoriteLCcheckbox",
                                                     label = "Land Cover at impact site:",
                                                     selected = LC_values,
                                                     choiceNames = LC_names,
                                                     choiceValues = LC_values)),
                           column(7,leafletOutput("meteoritemap")))),
              tabPanel("Meteorite Table",dataTableOutput('meteorite_table')),
              tabPanel("Land Cover PDFs",
                       fluidRow(
                           column(1,h3("")),
                           column(2,checkboxGroupInput(inputId = "pdfcheckbox",
                                                       label = "Choose Land Covers to display",
                                                       selected = LC_values,
                                                       choiceNames = LC_names,
                                                       choiceValues = LC_values)),
                           column(9,plotOutput("pdfplot")))
                      )
          )
    )
)                  

##SERVER
server <- function(input, output, session) {
    
    output$pdfplot = renderPlot({
        pdf_plot <- subset(pdfmelt, variable %in% input$pdfcheckbox)
        p <- ggplot()
        p <- p + geom_line(data = pdf_plot, aes(x = x_axis, y = value, color=variable, group=variable))
        p <- p + ggtitle("Probability Density Function of Mass by Land Cover Type")
        p <- p + labs(x = "Meteorite Mass (g)", y= NULL)
        p
    })
    
         output$meteorite_table = renderDataTable(meteorites)

map_meteorites <- meteorites
#         map_meteorites <- subset(map_meteorites, Fell_or_Found %in% input$fellfoundcheckbox)
#         map_meteorites <- map_meteorites[map_meteorites$Year >= input$yearslider[1] & map_meteorites$Year <= input$yearslider[2],]
#         map_meteorites <- map_meteorites[map_meteorites$Mass >= input$massslider[1] & map_meteorites$Mass <= input$massslider[2],]
#         map_meteorites <- subset(map_meteorites, Type %in% input$typecheckbox)
#         map_meteorites <- subset(map_meteorites, Chondrite %in% input$chondritecheckbox)
#         map_meteorites <- subset(map_meteorites, Land_Cover %in% input$meteoriteLCcheckbox)

    
    
    output$meteoritemap = renderLeaflet({
        map <- leaflet(map_meteorites) %>% addTiles() %>% addMarkers(clusterOptions = markerClusterOptions())
        map
    })
    
}
                                        
    
## Run App

shinyApp(ui = ui, server = server)

In [ ]:
output$meteoritemap = renderLeaflet({

    map_meteorites <- meteorites
    map_meteorites <- subset(map_meteorites, Fell_or_Found %in% input$fellfoundcheckbox)
    map_meteorites <- map_meteorites[map_meteorites$Year >= input$yearslider[1] & map_meteorites$Year <= input$yearslider[2],]
    map_meteorites <- map_meteorites[map_meteorites$Mass >= input$massslider[1] & map_meteorites$Mass <= input$massslider[2],]
    map_meteorites <- subset(map_meteorites, Type %in% input$typecheckbox)
    map_meteorites <- subset(map_meteorites, Chondrite %in% input$chondritecheckbox)
    map_meteorites <- subset(map_meteorites, Land_Cover %in% input$meteoriteLCcheckbox)

    map <- leaflet(map_meteorites) %>% addTiles() %>% addMarkers(clusterOptions = markerClusterOptions())
    map
    })


In [17]:
#DEPLOY TO SHINY SERVER
dir <- getwd() #This gets the current Working Directory
course <- "PSDS4900" #This is to specify the course path for the shiny server
folder <- "dashboard" #This specifies the folder name to copy

system(sprintf("/usr/local/bin/shiny_deploy %s %s %s", course, dir,folder), 
       intern = TRUE,
       ignore.stdout = FALSE, 
       ignore.stderr = FALSE,
       wait = TRUE, 
       input = NULL)

[1] "sending incremental file list"                                                                                       
[2] "dashboard/"                                                                                                          
[3] "dashboard/app.R"                                                                                                     
[4] "\r          6,097 100%    0.00kB/s    0:00:00  \r          6,097 100%    0.00kB/s    0:00:00 (xfr#1, to-chk=131/133)"
[5] ""                                                                                                                    
[6] "sent 10,026 bytes  received 56 bytes  20,164.00 bytes/sec"                                                           
[7] "total size is 84,131,891  speedup is 8,344.76"

In [2]:
url <- cat("https://shiny.sgn.missouri.edu/students/ymw.0780/",course,"/",folder,"\n\n",sep="")
print(url)

https://shiny.sgn.missouri.edu/students/ymw.0780/PSDS4900/dashboard

NULL


In [6]:
meteorites <- read.csv(file='data/results/meteorites_lc.csv')
meteorites <- meteorites[, c(2,3,7,8,6,10,9,18,13:16,12,5)]
colnames(meteorites) <- c('id','Meteorite Name','Fell_or_Found','Year', 'Mass (g)', 'Longitude', 'Latitude', 'Land Cover', 'Chondrite', 'Type', 'Level 3', 'Level 4', 'Group','recclass')

meteorites

id,Meteorite Name,Fell_or_Found,Year,Mass (g),Longitude,Latitude,Land Cover,Chondrite,Type,Level 3,Level 4,Group,recclass
1,Aachen,Fell,1880,21.0,6.08333,50.77500,Temperate Decidious Forest,Chondrite,Stony,Ordinary,L,L,L5
2,Aarhus,Fell,1951,720.0,10.23333,56.18333,Cultivated Land,Chondrite,Stony,Ordinary,H,H,H6
4,Abajo,Found,1982,331.0,-105.41667,26.80000,Pasture/Land used for Grazing,Chondrite,Stony,Ordinary,H,H,H5
5,Abbott,Found,1951,21100.0,-104.28333,36.30000,Grassland/Steppe,Chondrite,Stony,Ordinary,H,H,H3-6
6,Abee,Fell,1952,107000.0,-113.00000,54.21667,Cultivated Land,Chondrite,Stony,Enstatite,EH-EL,EH,EH4
7,Abernathy,Found,1941,2914.0,-101.80000,33.85000,Cultivated Land,Chondrite,Stony,Ordinary,L,L,L6
8,Abo,Found,1840,1.2,22.30000,60.43333,Cultivated Land,Chondrite,Stony,Ordinary,H,H,H
9,Abu Moharek,Found,1997,4500.0,29.83583,27.23944,Temperate Decidious Forest,Chondrite,Stony,Ordinary,H,H,H4
10,Acapulco,Fell,1976,1914.0,-99.90000,16.88333,Oceans/Water,Achondrite,Stony,Primitive,-,Acapulcoite,Acapulcoite
11,Acfer 001,Found,1989,6700.0,3.61667,27.50000,Hot Desert,Chondrite,Stony,Ordinary,L,L,L6


In [7]:
pdf <- read.csv(file = 'data/results/pdf.csv')
pdf

X,x_axis,water_pdf,cultivated_pdf,pasture_pdf,tundra_pdf,wooded_tundra_pdf,boreal_forest_pdf,conifer_forest_pdf,temperate_mixed_forest_pdf,temperate_decidous_forest_pdf,grasssland_pdf,desert_pdf,savanna_pdf,tropical_wooded_pdf,tropical_forest_pdf
0,0,8.336332e-06,6.120329e-07,1.951613e-07,4.926540e-06,1.032228e-05,3.046063e-06,7.760554e-08,3.048207e-06,5.896893e-06,1.750323e-07,2.347376e-07,5.564762e-07,2.474312e-05,3.502080e-05
1,1000,8.399062e-06,6.121160e-07,1.951685e-07,4.972965e-06,1.246673e-05,3.055230e-06,7.760904e-08,3.056104e-06,5.903189e-06,1.750406e-07,2.347491e-07,5.565957e-07,2.862377e-05,3.729027e-05
2,2000,8.458092e-06,6.121978e-07,1.951757e-07,5.018688e-06,1.486394e-05,3.064219e-06,7.761254e-08,3.063822e-06,5.908192e-06,1.750489e-07,2.347606e-07,5.567141e-07,3.257584e-05,3.925004e-05
3,3000,8.513339e-06,6.122780e-07,1.951828e-07,5.063684e-06,1.749522e-05,3.073029e-06,7.761603e-08,3.071360e-06,5.911899e-06,1.750571e-07,2.347720e-07,5.568315e-07,3.647208e-05,4.083756e-05
4,4000,8.564721e-06,6.123569e-07,1.951899e-07,5.107923e-06,2.032869e-05,3.081659e-06,7.761952e-08,3.078717e-06,5.914308e-06,1.750653e-07,2.347833e-07,5.569477e-07,4.017184e-05,4.200050e-05
5,5000,8.612167e-06,6.124343e-07,1.951969e-07,5.151381e-06,2.331866e-05,3.090105e-06,7.762300e-08,3.085891e-06,5.915418e-06,1.750735e-07,2.347945e-07,5.570629e-07,4.352906e-05,4.269965e-05
6,6000,8.655606e-06,6.125102e-07,1.952039e-07,5.194029e-06,2.640597e-05,3.098367e-06,7.762649e-08,3.092882e-06,5.915227e-06,1.750817e-07,2.348056e-07,5.571770e-07,4.640162e-05,4.291105e-05
7,7000,8.694975e-06,6.125847e-07,1.952108e-07,5.235843e-06,2.951922e-05,3.106444e-06,7.762996e-08,3.099687e-06,5.913735e-06,1.750898e-07,2.348167e-07,5.572900e-07,4.866125e-05,4.262743e-05
8,8000,8.730218e-06,6.126578e-07,1.952177e-07,5.276796e-06,3.257707e-05,3.114333e-06,7.763344e-08,3.106305e-06,5.910944e-06,1.750979e-07,2.348277e-07,5.574019e-07,5.020300e-05,4.185855e-05
9,9000,8.761282e-06,6.127294e-07,1.952246e-07,5.316863e-06,3.549143e-05,3.122033e-06,7.763692e-08,3.112736e-06,5.906856e-06,1.751059e-07,2.348386e-07,5.575127e-07,5.095330e-05,4.063070e-05


In [ ]:
library(reshape2)
pdfmelt<-melt(pdf,id="x_axis")
print(pdfmelt)

In [ ]:
library(ggplot2)

pdfcheckbox <- list("tropical_forest_pdf","water_pdf","tundra_pdf")

pdf_plot <- subset(pdfmelt, variable %in% pdfcheckbox)

p <- ggplot()
p <- p + geom_line(data = pdf_plot, aes(x = x_axis, y = value, color=variable, group=variable))
p <- p + ggtitle(paste0("Probability Density Function of Mass by Land Cover Type"))
p <- p + labs(x = "Meteorite Mass (g)", y= NULL)
p

pdf_plot

In [9]:
fellfoundcheckbox = c('Fell')
yearslider = c(500,1500)





map_meteorites <- meteorites
map_meteorites <- subset(map_meteorites, Fell_or_Found %in% fellfoundcheckbox)
map_meteorites <- map_meteorites[map_meteorites$Year >= yearslider[1] & map_meteorites$Year <= yearslider[2],]

map_meteorites

,id,Meteorite Name,Fell_or_Found,Year,Mass (g),Longitude,Latitude,Land Cover,Chondrite,Type,Level 3,Level 4,Group,recclass
3281,7823,Elbogen,Fell,1399,107000.0,12.73333,50.18333,Temperate Mixed Forest,Achondrite,Iron,Magmatic,Siderite,Iron IID,"Iron, IID"
3310,10039,Ensisheim,Fell,1491,127000.0,7.35000,47.86667,Temperate Decidious Forest,Chondrite,Stony,Ordinary,LL,LL,LL6
4974,16914,Narni,Fell,920,367.0,12.51667,42.51667,Temperate Decidious Forest,-,Stony,-,-,Stony,Stone-uncl
5038,16988,Nogata,Fell,860,472.0,130.75000,33.72500,Temperate Decidious Forest,Chondrite,Stony,Ordinary,L,L,L6
5614,22614,Rivolta de Bassi,Fell,1490,103.3,9.51667,45.48333,Cultivated Land,-,Stony,-,-,Stony,Stone-uncl
6699,24146,Valdinoce,Fell,1495,367.0,12.10000,44.06667,Pasture/Land used for Grazing,-,Stony,-,-,Stony,Stone-uncl
